In [7]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit

colors = [
'black','blue','red','green',
'cyan',  
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [8]:
# inject spikes
def generate_poisson_uniform_firingrate3(rec_len_sec, sample_rate, firing_rates):
   
    # 1ms wide bins used for now;
    bin_width = 0.001 # ms precise bins;
    
    # sample uniformly to get firing rate between 1-10Hz
    f_rate = np.random.rand()*firing_rates[1]+firing_rates[0]
    
    # set threshold for spiking for a poisson process
    poisson_thresh = f_rate*bin_width

    # generate random spike probabilities;
    # make sure last 10 milliseconds doesn't have any spikes
    spikes = np.random.rand(int(rec_len_sec/bin_width)-10)

    # times in milliseconds
    idx=np.where(spikes<=poisson_thresh)[0]

    # find inter-time interval and obtain scaling;
    # spikes > 100ms apart are not scaled
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(diffs>=100)[0]
    diffs[idx2]=100
    
    # set scale
    #scale = np.exp((diffs-100)/200)
    scale = np.exp((diffs-100)/1000)
    scale = np.hstack((1.0,scale))
    
    print ("TODO: implement randomized scaling")
      
    # convert from milliseconds to sample_rate sample-time + add random shift so 
    # not all spike trains land exactly on sample 0; 
    # Cat: TODO add individual spike-time shifts, and avoid refractory violations;
    #times = idx * sample_rate//1000 +np.random.randint(sample_rate//1000,size=1)
    times = idx * sample_rate//1000 + np.random.randint(sample_rate//1000,size=idx.shape[0])
    
    return (times, scale)

In [9]:
templates = np.load('/media/cat/1TB/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy')
print (templates.shape)

geom = np.loadtxt('/media/cat/1TB/data/synthetic/p1_g0_t0.imec0.ap_geom.txt')
geom = geom[:templates.shape[2]]
print (geom.shape)

(575, 101, 384)
(384, 2)


In [10]:
ptps = templates.ptp(1).max(1)
print ("ptps: ", ptps[:10])

idx = np.where(ptps>6)[0]
print (idx.shape)

templates = templates[idx]

max_chans = templates.ptp(1).argmax(1)
print ("max chans: ", max_chans[:10])

for k in range(384):
    idx = np.where(max_chans==k)[0]
    print (k, idx.shape[0])
    
    

ptps:  [12.802503  7.022945  9.097455  6.203224  9.204906  8.315739  8.175831
  9.523293  8.208408 13.281512]
(537,)
max chans:  [ 0  0  2  4  1  2  4  3  5 11]
0 2
1 1
2 2
3 1
4 2
5 1
6 1
7 1
8 1
9 3
10 1
11 5
12 1
13 3
14 0
15 0
16 0
17 2
18 2
19 0
20 1
21 0
22 1
23 2
24 1
25 1
26 2
27 1
28 2
29 0
30 1
31 1
32 2
33 0
34 0
35 0
36 2
37 1
38 0
39 1
40 4
41 1
42 1
43 1
44 1
45 3
46 1
47 0
48 2
49 1
50 0
51 2
52 0
53 1
54 2
55 1
56 1
57 1
58 2
59 1
60 5
61 1
62 2
63 1
64 0
65 1
66 3
67 1
68 3
69 7
70 1
71 0
72 1
73 2
74 2
75 0
76 2
77 1
78 2
79 2
80 1
81 5
82 0
83 2
84 0
85 1
86 0
87 1
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 1
121 0
122 0
123 1
124 0
125 2
126 0
127 0
128 0
129 0
130 2
131 2
132 1
133 2
134 2
135 3
136 5
137 5
138 0
139 4
140 6
141 9
142 2
143 4
144 4
145 8
146 4
147 4
148 4
149 5
150 10
151 6
152 3
153 4
154 3
155 5
156 7
157 3


In [11]:
# print (templates.shape)

# idxs = [ 52,  61,  62,  64, 295]
# print (idxs)


max_chans = templates.ptp(1).argmax(1)
# print (max_chans[idxs])

# plt.plot(templates[idxs,:,max_chans[idxs]].T)
# plt.show()

In [27]:
# select a channel with many templates on it
selected_chan = 141

import scipy, scipy.spatial
a = np.float32(geom[selected_chan])[None]
b = np.float32(geom)
print (a.shape, b.shape)


# find nearby channels and templates
dist = scipy.spatial.distance.cdist(a,b, 'euclidean') #
print ("dist: ", dist[:,:10])
all_chans = np.where(dist<50)[1]
print ("neighbour chans: ", all_chans)

fig=plt.figure()
ctr=0
times = []
injected_templates2 = []
for chan in all_chans:
    # select data
    idx = np.where(max_chans==chan)[0]
    print ("neighbour templates: ", idx, "  max chan: ", chan)
    if idx.shape[0]==0:
        continue

    injected_templates = templates[idx,:,selected_chan]
    #injected_templates = np.delete(injected_templates,3,axis=0)

    for k in range(injected_templates.shape[0]):
        plt.plot(injected_templates[k], c=colors[ctr], label = str(ctr*10+k))
    injected_templates2.extend(injected_templates)
    print (injected_templates.shape)

    # generate spike trains
    rec_len_sec = 300
    sample_rate = 30000

    firing_rates = [35,20]

    for k in range(injected_templates.shape[0]):
        times1, scale = generate_poisson_uniform_firingrate3(rec_len_sec, sample_rate, firing_rates)
        times.append(times1)
    ctr+=1
#print (times.shape)
plt.legend(fontsize=10)
print (times)
plt.show()
injected_templates2 = np.vstack(injected_templates2)

(1, 2) (384, 2)
dist:  [[1400.36566653 1400.         1380.09275051 1380.09275051 1360.3764185
  1360.         1340.09551898 1340.09551898 1320.38782182 1320.        ]]
neighbour chans:  [137 138 139 140 141 142 143 145]
neighbour templates:  [113 124 125 129 500]   max chan:  137
(5, 101)
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
neighbour templates:  []   max chan:  138
neighbour templates:  [126 130 136 501]   max chan:  139
(4, 101)
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
neighbour templates:  [121 128 131 132 133 139]   max chan:  140
(6, 101)
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
TODO: implement randomized scaling
nei

In [28]:

# add spikes to raw data
data_raw = np.zeros(sample_rate*rec_len_sec, 'float32')

# skip broadcast addtiion
# do single spike insertion at at me to avoid overwriting nearby spikes
for k in range(len(times)):
    #print (times[k][:,None]+np.arange(injected_templates.shape[1]))
    for p in range(times[k].shape[0]):
        data_raw[times[k][p]+np.arange(injected_templates2.shape[1])]+= injected_templates2[k]

fig=plt.figure()    
plt.plot(data_raw)

plt.show()

np.save('/media/cat/1TB/data/synthetic/ari2/data_raw_'+str(sample_rate)+"hz_"+str(rec_len_sec)+"sec_"+str(firing_rates[0])+"hz_to_"+str(firing_rates[0]+firing_rates[1])+"hz.npy",
       data_raw)
np.save('/media/cat/1TB/data/synthetic/ari2/data_raw_'+str(sample_rate)+"hz_"+str(rec_len_sec)+"sec_"+str(firing_rates[0])+"hz_to_"+str(firing_rates[0]+firing_rates[1])+"hz_times.npy",
       times)
np.save('/media/cat/1TB/data/synthetic/ari2/data_raw_'+str(sample_rate)+"hz_"+str(rec_len_sec)+"sec_"+str(firing_rates[0])+"hz_to_"+str(firing_rates[0]+firing_rates[1])+"hz_templates.npy",
       injected_templates2)

In [31]:
data = np.load('/media/cat/1TB/data/synthetic/ari2/data_raw_30000hz_300sec_5hz_to_25hz_templates.npy')
print (data.shape)
plt.plot(data.T)
plt.show()

(38, 101)
